# Connnecting to BigQuery

## <center> <font color=#DB1830>This document shows the recommended way to authenticate to a Cloud API</center> </font>  


#### Doc:  
#### https://cloud.google.com/docs/authentication/  
#### https://github.com/GoogleCloudPlatform/google-cloud-python/tree/master/bigquery
#### https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/reference.html  






#### GCP client libraries use a strategy called Application Default Credentials (ADC) to find your application's credentials. When your code uses a client library, the strategy checks for your credentials in the following order:

1. First, ADC checks to see if the environment variable **GOOGLE_APPLICATION_CREDENTIALS** is set. If the variable is set, ADC uses the service account file that the variable points to. The next section describes how to set the environment variable.

2. If the environment variable isn't set, ADC uses the default service account that Compute Engine, Kubernetes Engine, Cloud Run, App Engine, and Cloud Functions provide, for applications that run on those services.

3. If ADC can't use either of the above credentials, an error occurs.


#### How setting a Service Account and an Environment Variable
#### Doc: https://cloud.google.com/docs/authentication/getting-started/ 

##### Create a Service Account (GCP Project owners)
1. From Google Cloud Console select your project
2. Go to the Create service account key [page](https://console.cloud.google.com/iam-admin/serviceaccounts)
3. select New service account
4. Enter a name an a description: who the Service Account is related to (user, dept, project or application)
5. Create a Key, download the .json file with the Credentials and save the file on your laptop
6. From the Cloud Console select [IAM](https://console.cloud.google.com/iam-admin/) and assign a role to the Service Account

##### Create an Environment Variable (Windows users)
1. From Control Panel\System and Security\System select Advanced system settings
2. Select Environment Varibales
3. In the System variables section, create a New variable named **GOOGLE_APPLICATION_CREDENTIALS** and set the path to the .json file with the credentials


##### Verifying authentication
By default the credentials stored in the system variable GOOGLE_APPLICATION_CREDENTIALS are used:

In [5]:
from google.cloud import bigquery

project_id = 'bi-team-189611'
client_ref = bigquery.Client(project=project_id)

query_1 = '''
SELECT SESSION_USER() AS USER
'''

query_job = client_ref.query(query_1)
results = query_job.result()
for row in results:
    print(row)


Row(('py-account@bi-team-189611.iam.gserviceaccount.com',), {'USER': 0})


#### The credentials file can also be called explicitly:


In [7]:
from google.cloud import bigquery

credentials_file = 'C:/Dev/bq/Usage_Reporting_key/usage-data-reporting-0192446b1bb9.json'

project_id = 'usage-data-reporting'

# Explicitly use service account credentials by specifying the private key file
client_ref = bigquery.Client.from_service_account_json(credentials_file)

query_1 = '''
SELECT SESSION_USER() AS USER
'''

query_job = client_ref.query(query_1)
results = query_job.result()
for row in results:
    print(row)


Row(('py-account-ag@usage-data-reporting.iam.gserviceaccount.com',), {'USER': 0})


#### Example: creating a table in a GBQ dataset using a Pyhton script

In [9]:
#################################################################################################
# BigQuery: this script creates a table in a GBQ dataset
# ###############################################################################################
from google.cloud import bigquery
import datetime

# Define variables
project_id = 'bi-team-189611'
dataset_id = 'TR_freshdesk_LDZ'
table_id = 'groups3'

# Set GCP Credentials
key_path = 'C:/Dev/fd/bi-team-189611-c78f094e47ef.json'
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Set a client
client_ref = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

dataset_ref = client_ref.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

#Table Schema
table_schema = [
        bigquery.SchemaField('id', 'INTEGER', description='Group Id'),
        bigquery.SchemaField('created_at', 'TIMESTAMP', description='Group creation date'),
        bigquery.SchemaField('updated_at', 'TIMESTAMP', description='Group updated date'),
        bigquery.SchemaField('name', 'STRING', description='Group name'),
        bigquery.SchemaField('description', 'STRING', description='Group desc'),
        bigquery.SchemaField('escalate_to', 'INTEGER', description='Escalate to Group'),
        bigquery.SchemaField('group_type', 'STRING', description='Time Zone'),
        bigquery.SchemaField('auto_ticket_assign', 'BOOL', description='Auto assign'),
        bigquery.SchemaField('business_hour_id', 'INTEGER', description='Auto assign')
]

gbq_table = bigquery.Table(table_ref, schema=table_schema)
x = datetime.datetime.now()
mytable = client_ref.create_table(gbq_table) #API request
print(x, ' - Table created: ', mytable)

2020-01-27 11:43:39.082649  - Table created:  Table(TableReference(DatasetReference('bi-team-189611', 'TR_freshdesk_LDZ'), 'groups3'))
